# Лабораторная работа №3
## Выполнил студент группы БФИ2001 Маров Артур Константинович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
import time
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

import pandas

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [4]:
start_time = time.time()

string = input("Строка: ")
substring = input("Подстрока: ")
register = input("Включить чувствительность к регистру? (0 - нет, 1 - да): ")

#string = "лилилилась лилилось"
#substring = "лилила"
#print(string)
#print(substring)

def Knuth_Morris_Pratt_search(substring, string, register):
    if register == '0':
        string = string.lower()
        substring = substring.lower()
        
    m = len(substring)
    n = len(string)
 
    lps = [0]*m
    j = 0 
    compute_LPS_Array(substring, m, lps)
 
    i = 0 
    while i < n:
        if substring[j] == string[i]:
            i += 1
            j += 1
 
        if j == m:
            return (i-j)
            j = lps[j-1]
 
        elif i < n and substring[j] != string[i]:
            if j != 0:
                j = lps[j-1]
            else:
                i += 1
 
def compute_LPS_Array(substring, m, lps): #longest prefix suffix
    len = 0 
    lps[0] 
    i = 1
    while i < m:
        if substring[i]== substring[len]:
            len += 1
            lps[i] = len
            i += 1
        else:
            if len != 0:
                len = lps[len-1]
            else:
                lps[i] = 0
                i += 1
 
print("Первое вхождение:", Knuth_Morris_Pratt_search(substring, string, register))
time_kmp_search = round((time.time() - start_time)*1000)
print("--- {:.3f} ms ---".format(time_kmp_search))

Строка: лилилилась лилилось
Подстрока: Лилила
Включить чувствительность к регистру? (0 - нет, 1 - да): 0
Первое вхождение: 2
--- 20907.000 ms ---


#### Упрощенный алгоритм Бойера-Мура

In [8]:
start_time = time.time()

string = input("Строка: ")
substring = input("Подстрока: ")
register = input("Включить чувствительность к регистру? (0 - нет, 1 - да): ")
#substring = "лилилилась лилилось"
#string = "лилила"
#print(string)
#print(substring)

class last_occurrence(object):

    def __init__(self, pattern, alphabet):
        self.occurrences = defaultdict()
        for letter in alphabet:
            self.occurrences[letter] = pattern.rfind(letter)

    def __call__(self, letter):
        return self.occurrences[letter]

def Boyer_Moore_search(string, substring):
    #register_check(string, substring, register)
    if register == '0':
        string = string.lower()
        substring = substring.lower()
        
    alphabet = set(string)
    last = last_occurrence(substring, alphabet)
    m = len(substring)
    n = len(string)
    i = m - 1  
    j = m - 1  
    while i < n:
        if string[i] == substring[j]:
            if j == 0:
                return (i)
            else:
                i -= 1
                j -= 1
        else:
            l = last(string[i])
            i = i + m - min(j, 1+l)
            j = m - 1 
    return "Совпадений не найдено"#-1

print("Первое вхождение:", Boyer_Moore_search(string,substring))
time_bm_search = round((time.time() - start_time)*1000)
print("--- {:.3f} ms ---".format(time_bm_search))

Строка: лилилилась лилилось
Подстрока: Лилила
Включить чувствительность к регистру? (0 - нет, 1 - да): 0
Первое вхождение: 2
--- 9365.000 ms ---


Стандартная функция поиска

In [11]:
start_time = time.time()

sourcestr="Hello world"
substr="world"

print("Первое вхождение:", sourcestr.find(substr))
time_standard_search = round((time.time() - start_time)*1000)
print("--- {:.3f} ms ---".format(time_standard_search))

Первое вхождение: 6
--- 0.000 ms ---


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [9]:
from heapq import heappop, heappush
from math import sqrt
import argparse

counter = 0

def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

    raise Exception("No solution?!")

def manh_dst_matrix(a, b, n):
    # расстояние между `a` и `b` в матрице размером `n`
    
    return abs(a % n - b % n) + abs(a // n - b // n)

class chain15:
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size ** 2:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == 3:
                sstr += "\n"
            i += 1
        return sstr

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)

    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)
    
    def last_node(self):
        # должно быть хэшируемо (list not hashable :( )
        
        return str(self.board_state)
    
    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    def simple_heur(self):
        dst = 0
        for i in range(0, int(self.quad_size)):
            if (self.board_state[i] - 1) != i:
                dst += 1
        return dst

    def h(self):
        return self.manh_dst() + self.last_move()
    
    def g(self):
        return len(self.history)

    def f(self):
        return self.h() 

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # сравниваем с соседними элементами
        if zero_coord + 1 < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord + self.size < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        return neighs
    
start_state = [1, 0, 3, 4, 5, 6, 7, 10, 12, 8, 11, 9, 13, 14, 15, 2]
start = chain15(start_state)
end = chain15((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))

result = a_star(start, end.last_node())

for node in result.history:
    print(str(node))
    print("-------------------------")
    counter += 1
print(str(result))
print("Количество шагов", counter)

1 0 3 4 
5 6 7 10 
12 8 11 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 0 7 10 
12 8 11 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 0 11 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 0 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 9 0 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 9 2 
13 14 15 0 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 9 2 
13 14 0 15 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 0 2 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 2 0 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 7 0 
12 11 2 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 0 7 
12 11 2 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
12 11 0 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
12 0 11 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
0 12 11 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
13 12 11 10 
0 1

### Вывод

In [13]:
print("n — длина строки, m — длина подстроки:")
table_search = [["Алгоритм Кнута-Морриса-Пратта", time_kmp_search, "O(n*m)", "O(n+m)"],
              ["Упрощенный алгоритм Бойера-Мура", time_bm_search, "O(n+m)", "O(m)"], 
              ["Стандартная функция поиска", time_standard_search, "O((n-m)*m)", "O(m)"]]


pandas.DataFrame(table_search, columns=["Алгортим сортировки","Время работы алгоритма", "Сложность в худшем случае", "Сложность в лучшем случае"])

n — длина строки, m — длина подстроки:


,Алгортим сортировки,Время работы алгоритма,Сложность в худшем случае,Сложность в лучшем случае
0,Алгоритм Кнута-Морриса-Пратта,20907,O(n*m),O(n+m)
1,Упрощенный алгоритм Бойера-Мура,9365,O(n+m),O(m)
2,Стандартная функция поиска,0,O((n-m)*m),O(m)
